Веб-сервис: На основе корпуса ("Война и мир" или другие сочинения Льва Толстого http://digital.tolstoy.ru/tolstoy_search/data/ ) и марковской модели (например, как тут https://habrahabr.ru/post/88514/) генерирует предложения в ответ на реплику пользователя

In [1]:
import re
from random import uniform
from collections import defaultdict

In [57]:
def propers(corpus):
    with open (corpus, 'r', encoding='utf-8') as src:
        lines=src.readlines()
    proper_names=[]
    for line in lines:
        proper=re.findall(r'[a-яА-Я] [А-Я][а-я]+', line, flags=re.DOTALL)
    #print()
        for p in proper:
            proper_names.append(p[2:].lower())
    return proper_names

In [56]:
propers('oxxxy.txt')

['дарвина',
 'мишки',
 'иди',
 'хаменеи',
 'ватикана',
 'модильяни',
 'луи',
 'чемодан',
 'фердинанд',
 'луперкаль',
 'селин',
 'азии',
 'дон']

In [2]:
r_alphabet = re.compile(u'[а-яА-Я0-9-]+|[.,:;?!]+')

def gen_lines(corpus):
#    data = open(corpus)
    with open(corpus, 'r', encoding='utf-8') as src:
        data=src.readlines()
    for line in data:
        yield line.lower()


def gen_tokens(lines):
    for line in lines:
        for token in r_alphabet.findall(line):
            if 
            yield token

def gen_trigrams(tokens):
    t0, t1 = '$', '$'
    for t2 in tokens:
        yield t0, t1, t2
        if t2 in '.!?':
            yield t1, t2, '$'
            yield t2, '$','$'
            t0, t1 = '$', '$'
        else:
            t0, t1 = t1, t2


In [30]:
def train(corpus):
    lines = gen_lines(corpus)
    tokens = gen_tokens(lines)
    trigrams = gen_trigrams(tokens)
    #clear_tokens=[t for t in tokens if t != ',' and t != '-' and t != '...' and t!= ':' and t!= ';']
    #clear_trigrams = gen_trigrams(clear_tokens)
    #print(clear_trigrams)
    #for t in tokens:
    #    print(t)
    bi, tri = defaultdict(lambda: 0.0), defaultdict(lambda: 0.0)

    for t0, t1, t2 in trigrams:
        bi[t0, t1] += 1
        tri[t0, t1, t2] += 1
    #print(tri)
    model = {}
    for (t0, t1, t2), freq in tri.items():
        if (t0, t1) in model:
            model[t0, t1].append((t2, freq/bi[t0, t1]))
        else:
            model[t0, t1] = [(t2, freq/bi[t0, t1])]
    return model
train('oxxxy.txt')

{('!', '$'): [('$', 1.0)],
 ('$', '$'): [('был', 0.05555555555555555),
  ('вы', 0.027777777777777776),
  ('сын', 0.027777777777777776),
  ('лица', 0.027777777777777776),
  ('мир', 0.027777777777777776),
  ('и', 0.027777777777777776),
  ('в', 0.027777777777777776),
  ('клипы', 0.027777777777777776),
  ('не', 0.027777777777777776),
  ('нам', 0.027777777777777776),
  ('социальными', 0.027777777777777776),
  ('мы', 0.027777777777777776),
  ('что', 0.05555555555555555),
  ('океан', 0.027777777777777776),
  ('или', 0.027777777777777776),
  ('стекла', 0.027777777777777776),
  ('до', 0.027777777777777776),
  ('мало', 0.05555555555555555),
  ('тут', 0.027777777777777776),
  ('как', 0.027777777777777776),
  ('ух', 0.027777777777777776),
  ('пустота', 0.027777777777777776),
  ('пусть', 0.027777777777777776),
  ('.', 0.027777777777777776),
  ('скоро', 0.027777777777777776),
  ('вот', 0.027777777777777776),
  ('ты', 0.027777777777777776),
  ('правда', 0.027777777777777776),
  ('нас', 0.027777777777

In [4]:
def unirand(seq):
    sum_, freq_ = 0, 0
    for item, freq in seq:
        sum_ += freq
    rnd = uniform(0, sum_)
    for token, freq in seq:
        freq_ += freq
        if rnd < freq_:
            return token

In [72]:
a=input('put the sentence in: ')
a=a.lower()
a=a.split()
model=train('oxxxy.txt')
all_names=propers('oxxxy.txt')
#print(all_names)
if len(a)==1:
    t0, t1= '$', a[0]
    phrase=t1+' '
else:
    t0, t1= a[-2], a[-1]
#phrase=t0+' '+t1
    phrase=t0+' '+t1
while t1 != '$':
    #if t1 in all_names:
    #    t1=t1.capitalize()
    try:
        t0, t1=t1, unirand(model[t0, t1])
    except:
        t0, t1 = '$', '$'
        t0, t1=t1, unirand(model[t0, t1])
    #print (t1)
    #if t1 == '$': break
    if t1 in ('.!?,;:') or t0 == '$':
       # if t1 not in all_names:
       #     print('result 1')
        phrase += t1
        #else:
        #    print('result 2')
        #    t1_new=t1.capitalize()
        #    print(t1_new)
        #    phrase += t1_new + ' '
    else:
        if t1 != '$':
            if t1 not in all_names:
                print('result 3')
                phrase += ' ' + t1
            else:
                print('result 4')
                t1_new=t1.capitalize()
                print(t1_new)
                phrase += ' ' + t1_new
print(phrase.capitalize())

#unirand(model[a[-2], a[-1]])

put the sentence in: и ветра
result 3
result 3
result 4
Азии
И ветра дуют с азии.


In [84]:
def input_sentence():
    a=input('put the sentence in: ')
    a=a.lower()
    a=a.split()
    return a

model=train('oxxxy.txt')
all_names=propers('oxxxy.txt')

def gen_sent(inp, mod, n):
    phrase=''
    
    if len(inp)==1:
        t0, t1= '$', inp[0]
        phrase=t1+' '
    else:
        t0, t1= inp[-2], inp[-1]
        phrase=t0+' '+t1
    while t1 != '$':
        try:
            t0, t1=t1, unirand(model[t0, t1])
        except:
            t0, t1 = '$', '$'
            t0, t1=t1, unirand(model[t0, t1])
        if t1 in ('.!?,;:') or t0 == '$':
            phrase += t1
        else:
            if t1 != '$':
                if t1 not in n:
                    print('result 3')
                    phrase += ' ' + t1
                else:
                    print('result 4')
                    t1_new=t1.capitalize()
                    print(t1_new)
                    phrase += ' ' + t1_new
    #print(phrase.capitalize())
    return phrase.capitalize()

In [88]:
gen_sent(input_sentence(), model, all_names)

put the sentence in: в мире
result 3
result 3
result 4
Мишки
result 3
result 3
result 3


'В мире идиллий и мишки, наивны, как прозелит.'

In [44]:
def generate_sentence(model):
    phrase = ''
    t0, t1 = '$', '$'
    a=input('put the sentence in: ')
    a=a.lower()
    a=a.split()
    
    
    while 1:
    #if a:
        t0, t1 = t1, unirand(model[a[-2], a[-1]])
        print(t0)
        print('____')
        print(t1)
        print('-----------------------------')
        if t1 == '$': break
        if t1 in ('.!?,;:') or t0 == '$':
            phrase += t1
        else:
            phrase += ' ' + t1
    return phrase.capitalize()


In [ ]:
if __name__ == '__main__':
    model = train('oxxxy.txt')
    for i in range(10):
        print (generate_sentence(model))